<a href="https://colab.research.google.com/github/Thanh-Phuc/Type-2-Diabetes-Stroke-prediction/blob/main/Stroke_prediction_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit
import streamlit as st

# Installation

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
import streamlit as st
from streamlit_shap import st_shap
import shap
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle
import streamviz
import joblib

# Running the application

## Data loading

In [ ]:
@st.cache_data
def load_model():
    model_path = "./weight/stroke_diabetes_xgboost.pkl"
    xgb_model_loaded = pickle.load(open(model_path, "rb"))
    return xgb_model_loaded

## Impute missing

In [ ]:
# def impute_user_dataframe(df_beta):
#     print("df_beta",df_beta)
#     with open('./weight/mice_imputer.pkl', 'rb') as f:
#         mice_imputerX = pickle.load(f)
#
#     df_tmu = df_beta
#     # percentage of null in every column
#     null_percentage = df_tmu.isnull().sum() / df_tmu.shape[0] * 100
#     # list of columns having missing
#     col_missing_tmu = null_percentage[null_percentage > 0].keys()
#     # create df with missing columns
#     df_tmu_mice = df_tmu.filter(col_missing_tmu, axis=1).copy()
#     # Fill missing value
#     print("df_tmu_mice",df_tmu_mice)
#     df_tmu_mice_preimputed = pd.DataFrame(mice_imputerX.transform(df_tmu_mice), columns=df_tmu_mice.columns)
#     # df_tmu_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(df_tmu_mice), columns=df_tmu_mice.columns)
#     # Replace mising columns in ds1 by columns in df_mice
#     df_tmu[col_missing_tmu] = df_tmu_mice_preimputed[col_missing_tmu].values
#     df_tmu = df_tmu.apply(lambda col: np.where(0 > col, col.median(), col))
#
#     return df_tmu

## Desing apps

In [ ]:
st.set_page_config(layout="wide")

# CSS to style the sidebar and columns
st.markdown(
    """
    <style>
    .sidebar .sidebar-content {
        background-color: #333333;  /* Dark grey background */
        padding: 20px;
        border-radius: 10px;
    }
    .main-container {
        display: flex;
        justify-content: space-between;
    }
    .col {
        flex: 1;
        padding: 20px;
        margin: 10px;
        border-radius: 10px;
    }
    .col2 {
        background-color: #FFFFFF;  /* White background */
    }
    .col3 {
        background-color: #F0F2F6;  /* Light grey background */
    }
    .col4 {
        background-color: #F63366;  /* Primary color background */
    }
    </style>
    """,
    unsafe_allow_html=True
)

st.title("5-year Stroke Prediction among Type 2 Diabetes Patients")

# Sidebar content
with st.sidebar:
    st.markdown(
        """
        <div class='sidebar-content'>
        <h1 style='color: blue;'>Medical Data Entry</h1>
        <h2>Personal Information</h2>
        """,
        unsafe_allow_html=True
    )

    data = {
        'Age': 40,
        'BMI': 26.47704,
        'Cholesterol': 130.0,
        'Triglyceride': 200.0,
        'Creatinine': 0.5,
        'GlucoseAC': 70.0,
        'HbA1c': 4.6,
        'Antidiabetic drug': 9.0,
        'Sex': True,
        'Med_Beta_Blocking': False,
        'Med_Lipid_Modifying': False,
        'Nervous_System_Drugs': False,
        'Hyperlipidemia': False,
        'Med_Calcium_Channel_Blockers': False,
        'Cardiac_Therapy': False,
        'stroke_hist': False,
        'Med_Renin_Angiotensin': False,
        'Med_Antithrombotic': False,
        'Med_Antigout': False
    }

    age = st.number_input('Age', value=data['Age'], format="%i", step=1)
    sex = st.radio("Sex", ["Female", "Male"], horizontal=True, index=0)
    bmi = st.number_input('BMI', value=data['BMI'], format="%.1f", step=.1)
    stroke_hist = st.radio("Did you have Stroke before?", ["Never", "Yes"], horizontal=True, index=0)
    hyperlipidemia = st.radio("Do you currently have Hyperlipidemia?", ["Never", "Yes"], horizontal=True, index=0)

    st.subheader("Medication")
    Antidiabetic_baby = ["Metformin", "SU", "ALGi", "TZD",
                         "DPP4i", "GLP1-RA", "SGLT2i", "Other blood glucose lowering drugs, excl. insulins",
                         "Combination"]
    antidiabetic_option = st.selectbox("Antidiabetic drugs", Antidiabetic_baby)
    antidiabetic_drug = Antidiabetic_baby.index(antidiabetic_option)

    cardiac_therapy = st.radio("Cardiac therapy", ["No", "Yes"], horizontal=True, index=0)
    med_beta_blocking = st.radio("Beta blocking agents", ["No", "Yes"], horizontal=True, index=0)
    med_calcium_channel_blockers = st.radio("Calcium channel blockers", ["No", "Yes"], horizontal=True, index=0)
    med_renin_angiotensin = st.radio("Agents acting on the renin-angiotensin system", ["No", "Yes"], horizontal=True, index=0)
    med_lipid_modifying = st.radio("Lipid modifying agents", ["No", "Yes"], horizontal=True, index=0)
    med_antigout = st.radio("Antigout preparations", ["No", "Yes"], horizontal=True, index=0)
    med_antithrombotic = st.radio("Antithrombotic agents", ["No", "Yes"], horizontal=True, index=0)
    nervous_system_drugs = st.radio("Nervous system drug", ["No", "Yes"], horizontal=True, index=0)

    st.subheader("Laboratory Test")
    cholesterol = st.number_input('Cholesterol (mg/dl)', value=data['Cholesterol'], format="%.1f", step=.1)
    triglyceride = st.number_input('Triglyceride (mg/dl)', value=data['Triglyceride'], format="%.1f", step=.1)
    creatinine = st.number_input('Creatinine (mg/dl)', value=data['Creatinine'], format="%.1f", step=.1)
    glucose_ac = st.number_input('GlucoseAC (mg/dl)', value=data['GlucoseAC'], format="%.1f", step=.1)
    hba1c = st.number_input('HbA1c', value=data['HbA1c'], format="%.1f", step=.1)

    data.update({
        'Age': age,
        'BMI': bmi,
        'Cholesterol': cholesterol,
        'Triglyceride': triglyceride,
        'Creatinine': creatinine,
        'GlucoseAC': glucose_ac,
        'HbA1c': hba1c,
        'Antidiabetic drug': antidiabetic_drug,
        'Sex': float(sex == "Male"),
        'Med_Beta_Blocking': float(med_beta_blocking == "Yes"),
        'Med_Lipid_Modifying': float(med_lipid_modifying == "Yes"),
        'Nervous_System_Drugs': float(nervous_system_drugs == "Yes"),
        'Hyperlipidemia': float(hyperlipidemia == "Yes"),
        'Med_Calcium_Channel_Blockers': float(med_calcium_channel_blockers == "Yes"),
        'Cardiac_Therapy': float(cardiac_therapy == "Yes"),
        'stroke_hist': float(stroke_hist == "Yes"),
        'Med_Renin_Angiotensin': float(med_renin_angiotensin == "Yes"),
        'Med_Antithrombotic': float(med_antithrombotic == "Yes"),
        'Med_Antigout': float(med_antigout == "Yes"),
    })

# Create columns for the main content
col2, col3 = st.columns(2)

with col2:
    st.markdown(
        """
        <div class='col col2'>
        <h2 style='color: red;'>Stroke Prediction</h2>
        </div>
        """,
        unsafe_allow_html=True
    )

    df = pd.DataFrame([data])#impute_user_dataframe(pd.DataFrame([data]))
    model = load_model()
    outcome = model.predict_proba(df.head(1))

    with open("./weight/explainer.sav", 'rb') as f:
        loaded_explainer = joblib.load(f)

    explainer = loaded_explainer
    shap_values = explainer(df)

    st_shap(shap.plots.waterfall(shap_values[0], max_display=15), height=600)

    def get_stroke_percentage(score):
        if score < 0 or score > 0.2:
            raise ValueError("Score must be between 0 and 0.2")
        breakpoints = [(0.0, 0), (0.002, 25), (0.0038, 50), (0.0139, 75), (0.2, 100)]
        for i in range(len(breakpoints) - 1):
            (x0, y0), (x1, y1) = breakpoints[i], breakpoints[i + 1]
            if x0 <= score <= x1:
                return y0 + (score - x0) * (y1 - y0) / (x1 - x0)
        return breakpoints[-1][1]

    gauge_value = round(get_stroke_percentage(outcome[0][1]), 2)

    low_range = 25
    mid_range = 75
    low_color = "green"
    mid_color = "yellow"
    high_color = "red"
    ar_top = 100

    streamviz.gauge(
        gVal=gauge_value, gSize="LRG",
        gTitle="Risk of stroke in 5 years", gMode="gauge+number",
        grLow=low_range,
        grMid=mid_range,
        gcLow=low_color,
        gcMid=mid_color,
        gcHigh=high_color,
        arTop=ar_top
    )

with col3:
    pass


## Running the apps

In [ ]:
streamlit run app.py